In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matsim
import geopandas as gpd
from pyproj import Geod
import os
import pollutants
import networkx as nx
from haversine import haversine_vector, Unit

# Aggregate all emissions spatial units

In [3]:
iter_list = list(range(300, 400))
root_path = r'../../../../data/clean/freightEmissions/shp/'
scenario_types = ['Basic', 'CB', 'Van']
h3_res = 10

In [4]:
scenarios_gdf = {}
scenarios_emissions = {}

for scenario in scenario_types:
    scenario_unit_gdf = gpd.GeoDataFrame()  # Create an empty GeoDataFrame to store only the geometries
    scenario_unit_emissions = {}
    for i in iter_list:
        print('Processing scenario: ' + scenario + ' iteration: ' + str(i))
        # Load the data
        iter_gdf = gpd.read_file(os.path.join(root_path, scenario.lower(), str(i), f'h3res{h3_res}.geojson'))

        ''' Concatenate the geometries of the current iteration to the scenario GeoDataFrame '''
        scenario_unit_gdf = pd.concat([scenario_unit_gdf, iter_gdf[['h3_index', 'geometry']]])
        scenario_unit_gdf = scenario_unit_gdf.drop_duplicates(subset=['h3_index'])
        
        ''' Get the detailed emissions for each spatial unit '''
        for idx, row in iter_gdf.iterrows():
            unit_idx = row['h3_index']
            detailed_emissions = (row[list(filter(lambda x: x not in ['h3_index', 'geometry'],iter_gdf.columns))]).to_dict()

            if scenario_unit_emissions.get(unit_idx) is None:  # If this is the first time we encounter this spatial unit
                scenario_unit_emissions[unit_idx] = {i: detailed_emissions}
            else:  # If this is not the first time we encounter this spatial unit, add the emissions to the existing dictionary
                scenario_unit_emissions[unit_idx][i] = detailed_emissions

    scenarios_gdf[scenario] = scenario_unit_gdf
    scenarios_emissions[scenario] = scenario_unit_emissions


Processing scenario: Basic iteration: 300
Processing scenario: Basic iteration: 301
Processing scenario: Basic iteration: 302
Processing scenario: Basic iteration: 303
Processing scenario: Basic iteration: 304
Processing scenario: Basic iteration: 305
Processing scenario: Basic iteration: 306
Processing scenario: Basic iteration: 307
Processing scenario: Basic iteration: 308
Processing scenario: Basic iteration: 309
Processing scenario: Basic iteration: 310
Processing scenario: Basic iteration: 311
Processing scenario: Basic iteration: 312
Processing scenario: Basic iteration: 313
Processing scenario: Basic iteration: 314
Processing scenario: Basic iteration: 315
Processing scenario: Basic iteration: 316
Processing scenario: Basic iteration: 317
Processing scenario: Basic iteration: 318
Processing scenario: Basic iteration: 319
Processing scenario: Basic iteration: 320
Processing scenario: Basic iteration: 321
Processing scenario: Basic iteration: 322
Processing scenario: Basic iterati

In [5]:
def cal_new_indicator(emission_df: pd.DataFrame, indicator_info: dict):
    copy_df = emission_df.copy(deep=True)
    # Calculate the new indicator
    for indicator, emissions_and_weights in indicator_info.items():
        filtered_emissions_and_weights = {k: v 
                                          for k, v in emissions_and_weights.items() if k in copy_df.iloc[0, 0].keys()}
        filtered_emissions = set(emissions_and_weights.keys()).difference(set(filtered_emissions_and_weights.keys()))
        if len(filtered_emissions) > 0:
            print(f" {filtered_emissions} is/are not found in the emissions df, and thus will not be considered in the calculation of the indicator")
        copy_df[indicator+'_mean'] = copy_df.apply(lambda x: np.mean([np.sum([x[i][sub_emission] * weight 
                                                                              for sub_emission, weight in filtered_emissions_and_weights.items()])
                                                                     for i in iter_list if isinstance(x[i], dict)]), 
                                                   axis=1)
        copy_df[indicator+'_median'] = copy_df.apply(lambda x: np.median([np.sum([x[i][sub_emission] * weight
                                                                                  for sub_emission, weight in filtered_emissions_and_weights.items()])
                                                                         for i in iter_list if isinstance(x[i], dict)]), 
                                                     axis=1) 

    return copy_df

In [6]:
def stats_unit(agg_emission_df: pd.DataFrame,
              spatial_unit_gdf: gpd.GeoDataFrame,
              emissions: list,  # A list of Emission component that we want to calculate the statistics for
              new_stats_index: dict = None,  # A dictionary of new statistics to be calculated {str'index_name': dict{sub_emission:weight}}
            #   stat_type: str,  # Statistic type: Mean or Median
              ):
    # assert stat_type.lower() in ['mean', 'median', 'both'], 'Invalid statistic type. Please choose between Mean and Median'
    assert isinstance(emissions, list), 'Pollutant must be a list'

    copy_agg_emission_df = agg_emission_df.copy(deep=True)
    copy_spatial_unit_gdf = spatial_unit_gdf.copy(deep=True)

    for e in emissions:
        assert e in pollutants.POLLUTANTS or e == 'air_quality_pollutants', 'Invalid pollutant'
        if e not in agg_emission_df.iloc[0,0].keys():
            print(f'{e} is not in the emissions dictionary. Skipping...')
            continue
        copy_agg_emission_df[e+'_mean'] = agg_emission_df.apply(lambda x: np.mean([x[i][e] 
                                                            for i in iter_list if isinstance(x[i], dict)]
                                                            ), 
                                            axis=1)

        copy_agg_emission_df[e+'_median'] = agg_emission_df.apply(lambda x: np.median([x[i][e] 
                                                            for i in iter_list if isinstance(x[i], dict)]
                                                            ), 
                                            axis=1)
    
        ''' Merge the statistics with the spatial unit GeoDataFrame '''
        copy_spatial_unit_gdf = pd.merge(copy_spatial_unit_gdf, copy_agg_emission_df[[e+'_mean', e+'_median']],
                                         left_on='h3_index', right_index=True)
    
    ''' Statistics for new indicators '''
    if new_stats_index is not None:
        new_indicator_agg_emission_df = cal_new_indicator(agg_emission_df, new_stats_index)
        for indicator in new_stats_index.keys():
            copy_spatial_unit_gdf = pd.merge(copy_spatial_unit_gdf, new_indicator_agg_emission_df[[indicator+'_mean', indicator+'_median']],
                                             left_on='h3_index', right_index=True)


    return copy_spatial_unit_gdf
    
    

In [176]:
scenario_stats_gdf = {}
for scenario in scenario_types:
    scenario_stats_gdf[scenario] = stats_unit(
        agg_emission_df=pd.DataFrame(scenarios_emissions[scenario]).T,
        spatial_unit_gdf=scenarios_gdf[scenario],
        emissions=[
            pollutants.CO,
            pollutants.SO2,
            pollutants.NOx,
            pollutants.NO2,
            pollutants.PM2_5,
            pollutants.PM2_5_non_exhaust,
            pollutants.PM,
            pollutants.PM_non_exhaust,
            "air_quality_pollutants",
        ],
        new_stats_index={
            "EPI": {
                pollutants.CO: 2.9,
                pollutants.SO2: 2.9,
                pollutants.NOx: 5.9,
                pollutants.NO2: 5.9,
                pollutants.PM2_5: 38.2,
                pollutants.PM2_5_non_exhaust: 38.2,
            },
            "weighted_AQI": {
                pollutants.SO2: 1,
                pollutants.NO2: 2.5,
                pollutants.PM: 5,
                pollutants.PM_non_exhaust: 5,
                pollutants.PM2_5: 10,
                pollutants.PM2_5_non_exhaust: 10,
            }

        },
    )

PM2_5_non_exhaust is not in the emissions dictionary. Skipping...
 {'PM2_5_non_exhaust'} is/are not found in the emissions df, and thus will not be considered in the calculation of the indicator
 {'PM2_5_non_exhaust'} is/are not found in the emissions df, and thus will not be considered in the calculation of the indicator
PM2_5_non_exhaust is not in the emissions dictionary. Skipping...
 {'PM2_5_non_exhaust'} is/are not found in the emissions df, and thus will not be considered in the calculation of the indicator
 {'PM2_5_non_exhaust'} is/are not found in the emissions df, and thus will not be considered in the calculation of the indicator


In [47]:
scenario_stats_gdf['Basic']

,h3_index,geometry,CO_mean,CO_median,SO2_mean,SO2_median,NOx_mean,NOx_median,NO2_mean,NO2_median,...,PM_mean,PM_median,PM_non_exhaust_mean,PM_non_exhaust_median,air_quality_pollutants_mean,air_quality_pollutants_median,EPI_mean,EPI_median,weighted_AQI_mean,weighted_AQI_median
0,8a1fa47ae9b7fff,"POLYGON ((173981.036 173422.746, 173965.942 17...",0.200103,0.218949,0.000248,0.000269,0.009202,0.009978,0.000460,0.000499,...,0.000491,0.000542,0.007883,0.008541,0.229210,0.250500,0.704218,0.769370,0.060596,0.065732
1,8a1fa47a0257fff,"POLYGON ((172113.226 173719.711, 172098.127 17...",0.100403,0.085247,0.000180,0.000153,0.007097,0.006025,0.000355,0.000301,...,0.000105,0.000089,0.005414,0.004596,0.122441,0.103960,0.387482,0.328994,0.042227,0.035853
2,8a1fa47a1887fff,"POLYGON ((172911.124 174813.711, 172896.028 17...",3.945672,3.526850,0.000255,0.000234,0.251416,0.224234,0.012571,0.011212,...,0.000266,0.000249,0.006009,0.005492,5.972117,5.332458,13.049822,11.675583,0.075918,0.073603
3,8a1fa47a02effff,"POLYGON ((172151.554 173608.968, 172136.454 17...",2.856087,3.561936,0.000383,0.000378,0.183859,0.228690,0.009193,0.011435,...,0.000200,0.000189,0.010324,0.009756,4.296479,5.379921,9.519857,11.830169,0.101398,0.103003
4,8a1fa47124f7fff,"POLYGON ((173702.040 175587.892, 173686.947 17...",0.204622,0.151654,0.000327,0.000239,0.013164,0.009336,0.000658,0.000467,...,0.000387,0.000377,0.010576,0.007553,0.244456,0.177591,0.765443,0.549439,0.080224,0.057271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,8a1fa47a0acffff,"POLYGON ((171921.597 174273.394, 171906.498 17...",0.074471,0.074471,0.000149,0.000149,0.027332,0.027332,0.001367,0.001367,...,0.000081,0.000081,0.004216,0.004216,0.113778,0.113778,0.420385,0.420385,0.034127,0.034127
110,8a1fa47a0347fff,"POLYGON ((171998.247 174051.927, 171983.148 17...",0.151452,0.151452,0.000247,0.000247,0.009505,0.009505,0.000475,0.000475,...,0.000259,0.000259,0.007898,0.007898,0.181274,0.181274,0.567542,0.567542,0.060213,0.060213
135,8a1fa47a0377fff,"POLYGON ((171876.300 174064.337, 171861.200 17...",0.064061,0.064061,0.000092,0.000092,0.003350,0.003350,0.000167,0.000167,...,0.000167,0.000167,0.003174,0.003174,0.074737,0.074737,0.230641,0.230641,0.023460,0.023460
178,8a1fa47a035ffff,"POLYGON ((172036.573 173941.190, 172021.473 17...",0.085395,0.085395,0.000153,0.000153,0.006036,0.006036,0.000302,0.000302,...,0.000089,0.000089,0.004604,0.004604,0.104139,0.104139,0.329562,0.329562,0.035915,0.035915


# Stats of road network metrics at spatial unit 

## Read the network for different scenarios

In [51]:
network_root_path = r'../../../../data/intermediate/test/freightEmissions/' 

bike_network = matsim.read_network(network_root_path + 'bikeGemeenteLeuvenWithHbefaType.xml.gz')
car_network = matsim.read_network(network_root_path + 'carGemeenteLeuvenWithHbefaType.xml.gz')

bike_network_gdf = bike_network.as_geo()
car_network_gdf = car_network.as_geo()

bike_network_gdf.crs = 'epsg:31370'
car_network_gdf.crs = 'epsg:31370'

bike_nodes = bike_network.nodes
car_nodes = car_network.nodes

bike_nodes_gdf = gpd.GeoDataFrame(bike_nodes, geometry=gpd.points_from_xy(bike_nodes.x, bike_nodes.y))
bike_nodes_gdf.crs = 'epsg:31370'
car_nodes_gdf = gpd.GeoDataFrame(car_nodes, geometry=gpd.points_from_xy(car_nodes.x, car_nodes.y))
car_nodes_gdf.crs = 'epsg:31370'

In [57]:
bike_network_gdf.to_file(r'../../../../data/clean/freightEmissions/shp/bike_network.geojson', driver='GeoJSON')
car_network_gdf.to_file(r'../../../../data/clean/freightEmissions/shp/car_network.geojson', driver='GeoJSON')   
bike_nodes_gdf.to_file(r'../../../../data/clean/freightEmissions/shp/bike_nodes.geojson', driver='GeoJSON')
car_nodes_gdf.to_file(r'../../../../data/clean/freightEmissions/shp/car_nodes.geojson', driver='GeoJSON')

## Spatial Join the Network with Spatial Unit

To get the number of links and nodes in each unit

In [54]:
def agg_unit_network_links_nodes(spatial_unit_gdf: gpd.GeoDataFrame, network_gdf: gpd.GeoDataFrame, nodes_gdf: gpd.GeoDataFrame):
    
    new_gdf = spatial_unit_gdf.copy(deep=True)
    
    ''' agg links '''
    sjoin_links = gpd.sjoin(new_gdf, network_gdf, how='left', predicate='intersects')
    link_stats = sjoin_links.groupby('h3_index').agg({'link_id': 'count'}).reset_index().rename(columns={'link_id': 'link_count'})
    # Merge the statistics with the spatial unit GeoDataFrame
    new_gdf = pd.merge(new_gdf, link_stats, on='h3_index', how='left')

    ''' agg nodes '''
    sjoin_nodes = gpd.sjoin(new_gdf, nodes_gdf, how='left', predicate='contains')
    node_stats = sjoin_nodes.groupby('h3_index').agg({'node_id': 'count'}).reset_index().rename(columns={'node_id': 'node_count'})
    # Merge the statistics with the spatial unit GeoDataFrame
    new_gdf = pd.merge(new_gdf, node_stats, on='h3_index', how='left')

    return new_gdf

## Agg nodes/link count 

In [177]:
for scenario, stats_gdf in scenario_stats_gdf.items():
    if scenario == "CB":
        network_gdf = bike_network_gdf
        nodes_gdf = bike_nodes_gdf
    else:
        network_gdf = car_network_gdf
        nodes_gdf = car_nodes_gdf
    scenario_stats_gdf[scenario] = agg_unit_network_links_nodes(stats_gdf, network_gdf, nodes_gdf)
    

## Convert network to graph

In [9]:
''' Convert network to graph '''
def getNodeInfo(nodesDict,
                nodeCoords,
                ):
    nodesDictKeys = list(nodesDict.keys())
    nodesDictValues = list(nodesDict.values())

    node_id = None
    if nodesDict == {}:
        return node_id

    # If the node is in the nodeDict
    if nodeCoords in nodesDictValues:
        node_id = nodesDictKeys[nodesDictValues.index(nodeCoords)]

    else:  # Find the 'same' node in the nodeDict (distance < 0.2m)
        dist_array = haversine_vector(np.array(nodesDictValues),
                                      np.array([nodeCoords]),
                                      Unit.METERS,
                                      comb=True)
        # Get the index of the 'same' node (if applicable)
        try:
            closeNode_idx = np.where(dist_array < 0.2)[1][0]
        except:
            closeNode_idx = None

        # If the point is close to an existing node, return the node id of the existing node
        if closeNode_idx is not None:
            node_id = nodesDictKeys[closeNode_idx]

    return node_id

def create_graph(road_gdf: gpd.GeoDataFrame,
                 length_col='length',
                 
                 ):
    splitG = nx.DiGraph()

    # Create a dictionary to store the unique nodes (id, coords)
    nodesDict = {}

    start_nodes = []
    end_nodes = []

    node_idx = 0
    for idx, row in road_gdf.iterrows():
        # idx = row.index
        length = row[length_col]
        line = row['geometry']
        line_orig_id = row['link_id']

        # (lat, lon) format
        lineStartCoords = (line.coords[0][1], line.coords[0][0])
        lineEndCoords = (line.coords[-1][1], line.coords[-1][0])

        # Get the node id of the start and end nodes
        startNodeOrigId = row['from_node']
        endNodeOrigId = row['to_node']

        startNode_id = getNodeInfo(nodesDict, lineStartCoords)
        if startNode_id is None:
            startNode_id = node_idx
            node_idx += 1
            nodesDict[startNode_id] = lineStartCoords

        endNode_id = getNodeInfo(nodesDict, lineEndCoords)
        if endNode_id is None:
            endNode_id = node_idx
            node_idx += 1
            nodesDict[endNode_id] = lineEndCoords

        start_nodes.append(startNode_id)
        end_nodes.append(endNode_id)

        # Add nodes to the graph
        if not splitG.has_node(startNode_id):
            splitG.add_node(startNode_id, x=lineStartCoords[1], y=lineStartCoords[0], 
                            orig_id=startNodeOrigId # Add the original node id of the node
                            )
        if not splitG.has_node(endNode_id):
            splitG.add_node(endNode_id, x=lineEndCoords[1], y=lineEndCoords[0],
                            orig_id=endNodeOrigId # Add the original node id of the node
                            )
        splitG.add_edge(startNode_id, endNode_id,
                        length=length,
                        orig_id=line_orig_id,
                        idx=idx)

    splitG.graph['crs'] = "EPSG:4326"

    indexed_road = road_gdf.copy()

    indexed_road['start_node'] = start_nodes
    indexed_road['end_node'] = end_nodes

    return splitG, indexed_road

In [34]:
bike_splitG, bike_indexed_road = create_graph(bike_network_gdf.to_crs('EPSG:4326'))
car_splitG, car_indexed_road = create_graph(car_network_gdf.to_crs('EPSG:4326'))

## identify intersections

In [30]:
def identify_intersections(graph: nx.DiGraph):
    intersections = {}

    for node in graph.nodes:
        
        if len(list(graph.neighbors(node))) >= 3:
            intersections[graph.nodes[node]['orig_id']] = list(graph.neighbors(node))
    intersection_df = pd.DataFrame(intersections.items(), columns=['orig_node_id', 'neighbors'])

    return intersection_df

In [37]:
bike_intersections = identify_intersections(bike_splitG)
bike_intersections

,orig_node_id,neighbors
0,1156293852,"[7, 42928, 42916]"
1,444420192,"[11, 25757, 9756]"
2,2065290220,"[21, 13963, 13962]"
3,1156955699,"[34, 32, 15507, 25755]"
4,1156955728,"[33, 15368, 40190]"
...,...,...
12749,1156192320,"[42904, 160, 42893]"
12750,1156192290,"[42900, 42915, 42914]"
12751,1156192329,"[42899, 42914, 42913]"
12752,1156192316,"[42906, 42912, 42898]"


In [43]:
bike_intersections_gdf = bike_intersections.merge(bike_nodes, left_on='orig_node_id', right_on='node_id', how='left')
bike_intersections_gdf = gpd.GeoDataFrame(bike_intersections_gdf, geometry=gpd.points_from_xy(bike_intersections_gdf.x, bike_intersections_gdf.y))
bike_intersections_gdf.crs = 'epsg:31370'
bike_intersections_gdf = bike_intersections_gdf[['orig_node_id', 'neighbors', 'geometry']]
bike_intersections_gdf

,orig_node_id,neighbors,geometry
0,1156293852,"[7, 42928, 42916]",POINT (174065.630 177001.873)
1,444420192,"[11, 25757, 9756]",POINT (174203.394 177709.610)
2,2065290220,"[21, 13963, 13962]",POINT (174449.916 177471.586)
3,1156955699,"[34, 32, 15507, 25755]",POINT (174872.324 171305.147)
4,1156955728,"[33, 15368, 40190]",POINT (174878.277 171307.623)
...,...,...,...
12749,1156192320,"[42904, 160, 42893]",POINT (173815.047 175839.662)
12750,1156192290,"[42900, 42915, 42914]",POINT (173943.531 175883.447)
12751,1156192329,"[42899, 42914, 42913]",POINT (173924.728 175881.180)
12752,1156192316,"[42906, 42912, 42898]",POINT (173907.900 175882.204)


In [36]:
car_intersections = identify_intersections(car_splitG)
car_intersections

,orig_node_id,neighbors
0,1156293852,"[1, 27037, 27025]"
1,1156595483,"[4, 3820, 16983]"
2,71365722,"[21, 25516, 25515]"
3,1088339997,"[31, 3085, 3084, 13969]"
4,1088339799,"[30, 25432, 25428]"
...,...,...
4449,1156192320,"[27010, 27008, 27023]"
4450,1156192290,"[27006, 27022, 27021]"
4451,1156192329,"[27005, 27021, 27020]"
4452,1156192316,"[27012, 27019, 27004]"


In [44]:
car_intersections_gdf = car_intersections.merge(car_nodes, left_on='orig_node_id', right_on='node_id', how='left')
car_intersections_gdf = gpd.GeoDataFrame(car_intersections_gdf, geometry=gpd.points_from_xy(car_intersections_gdf.x, car_intersections_gdf.y))
car_intersections_gdf.crs = 'epsg:31370'
car_intersections_gdf = car_intersections_gdf[['orig_node_id', 'neighbors', 'geometry']]
car_intersections_gdf

,orig_node_id,neighbors,geometry
0,1156293852,"[1, 27037, 27025]",POINT (174065.630 177001.873)
1,1156595483,"[4, 3820, 16983]",POINT (174832.403 179536.386)
2,71365722,"[21, 25516, 25515]",POINT (170654.689 174113.754)
3,1088339997,"[31, 3085, 3084, 13969]",POINT (173574.402 172841.118)
4,1088339799,"[30, 25432, 25428]",POINT (173561.096 172811.645)
...,...,...,...
4449,1156192320,"[27010, 27008, 27023]",POINT (173815.047 175839.662)
4450,1156192290,"[27006, 27022, 27021]",POINT (173943.531 175883.447)
4451,1156192329,"[27005, 27021, 27020]",POINT (173924.728 175881.180)
4452,1156192316,"[27012, 27019, 27004]",POINT (173907.900 175882.204)


### Agg the intersections stats into the stats_gdf

In [77]:
def agg_unit_network_intersections(spatial_unit_gdf: gpd.GeoDataFrame, intersections_gdf: gpd.GeoDataFrame):
    new_gdf = spatial_unit_gdf.copy(deep=True)
    
    ''' agg intersections '''
    sjoin_intersections = gpd.sjoin(new_gdf, intersections_gdf, how='left', predicate='contains')
    intersection_stats = sjoin_intersections.groupby('h3_index').agg({'orig_node_id': 'count'}).reset_index().rename(columns={'orig_node_id': 'intersection_count'})
    # Merge the statistics with the spatial unit GeoDataFrame
    new_gdf = pd.merge(new_gdf, intersection_stats, on='h3_index', how='left')
    new_gdf['intersections_ratio'] = new_gdf['intersection_count'] / new_gdf['node_count']
    return new_gdf

In [178]:
for scenario, stats_gdf in scenario_stats_gdf.items():
    if scenario == "CB":
        intersections_gdf = bike_intersections_gdf
    else:
        intersections_gdf = car_intersections_gdf
    scenario_stats_gdf[scenario] = agg_unit_network_intersections(stats_gdf, intersections_gdf)

## Network density

In [70]:
def geodesic_area(geom):
    geod = Geod(ellps="WGS84")
    if geom.is_empty:
        return np.nan
    try:
        area, _ = geod.geometry_area_perimeter(geom)
        return abs(area) / (1000 * 1000)
    except Exception as e:
        print(f"fail: {e}")
        return np.nan

In [73]:
def cal_unit_network_density(spatial_unit_gdf: gpd.GeoDataFrame, network_gdf: gpd.GeoDataFrame):
    new_gdf = spatial_unit_gdf.copy(deep=True)
    if 'area' not in new_gdf.columns:
        new_gdf['area'] = new_gdf['geometry'].area / (1000 * 1000)  # Convert to km2
    ''' Calculate the density of the network '''
    sjoin_links = gpd.sjoin(new_gdf, network_gdf, how='left', predicate='intersects')
    link_stats = sjoin_links.groupby('h3_index').agg({'length': 'sum'}).reset_index().rename(columns={'length': 'link_length'})
    # Merge the statistics with the spatial unit GeoDataFrame
    new_gdf = pd.merge(new_gdf, link_stats, on='h3_index', how='left')
    new_gdf['link_length'] = new_gdf['link_length'] / 1000  # Convert to km
    
    new_gdf['network_density'] = new_gdf['link_length'] / new_gdf['area']

    return new_gdf

In [179]:
for scenario, stats_gdf in scenario_stats_gdf.items():
    if scenario == "CB":
        network_gdf = bike_network_gdf
    else:
        network_gdf = car_network_gdf
    scenario_stats_gdf[scenario] = cal_unit_network_density(stats_gdf, network_gdf)

In [180]:
scenario_stats_gdf['Basic']

,h3_index,geometry,CO_mean,CO_median,SO2_mean,SO2_median,NOx_mean,NOx_median,NO2_mean,NO2_median,...,EPI_median,weighted_AQI_mean,weighted_AQI_median,link_count,node_count,intersection_count,intersections_ratio,area,link_length,network_density
0,8a1fa47ae9b7fff,"POLYGON ((173981.036 173422.746, 173965.942 17...",0.200103,0.218949,0.000248,0.000269,0.009202,0.009978,0.000460,0.000499,...,0.769370,0.060596,0.065732,24,19,0,0.000000,0.013034,0.567056,43.506402
1,8a1fa47a0257fff,"POLYGON ((172113.226 173719.711, 172098.127 17...",0.100403,0.085247,0.000180,0.000153,0.007097,0.006025,0.000355,0.000301,...,0.328994,0.042227,0.035853,2,0,0,NaN,0.013030,0.353569,27.135869
2,8a1fa47a1887fff,"POLYGON ((172911.124 174813.711, 172896.028 17...",3.945672,3.526850,0.000255,0.000234,0.251416,0.224234,0.012571,0.011212,...,11.675583,0.075918,0.073603,27,11,2,0.181818,0.013029,0.690478,52.997329
3,8a1fa47a02effff,"POLYGON ((172151.554 173608.968, 172136.454 17...",2.856087,3.561936,0.000383,0.000378,0.183859,0.228690,0.009193,0.011435,...,11.830169,0.101398,0.103003,28,9,2,0.222222,0.013030,1.024558,78.631154
4,8a1fa47124f7fff,"POLYGON ((173702.040 175587.892, 173686.947 17...",0.204622,0.151654,0.000327,0.000239,0.013164,0.009336,0.000658,0.000467,...,0.549439,0.080224,0.057271,36,11,4,0.363636,0.013028,1.660887,127.483592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,8a1fa47a0acffff,"POLYGON ((171921.597 174273.394, 171906.498 17...",0.074471,0.074471,0.000149,0.000149,0.027332,0.027332,0.001367,0.001367,...,0.420385,0.034127,0.034127,21,10,1,0.100000,0.013028,0.585219,44.920300
460,8a1fa47a0347fff,"POLYGON ((171998.247 174051.927, 171983.148 17...",0.151452,0.151452,0.000247,0.000247,0.009505,0.009505,0.000475,0.000475,...,0.567542,0.060213,0.060213,15,6,1,0.166667,0.013029,0.549267,42.158566
461,8a1fa47a0377fff,"POLYGON ((171876.300 174064.337, 171861.200 17...",0.064061,0.064061,0.000092,0.000092,0.003350,0.003350,0.000167,0.000167,...,0.230641,0.023460,0.023460,10,3,1,0.333333,0.013028,0.672172,51.593116
462,8a1fa47a035ffff,"POLYGON ((172036.573 173941.190, 172021.473 17...",0.085395,0.085395,0.000153,0.000153,0.006036,0.006036,0.000302,0.000302,...,0.329562,0.035915,0.035915,0,0,0,NaN,0.013029,0.000000,0.000000


## Depots 

In [84]:
depots_gdf = gpd.read_file(r'../../../../data/clean/freightEmissions/shp/depots/depots.shp')
depots_gdf

,link_id,to_node,x,y,node_id,geometry
0,10149534_2,83710713,173761.225525,174918.879277,83710713,POINT (173761.226 174918.879)
1,131757263_1-131757263_2,2632545926,172999.247657,173005.358094,2632545926,POINT (172999.248 173005.358)
2,150056709_r_10,1629894780,172603.874636,175121.079369,1629894780,POINT (172603.875 175121.079)
3,25806807_r_1,1640222126,173417.612182,175785.544374,1640222126,POINT (173417.612 175785.544)
4,27566523_11,87913344,175138.453974,175085.460190,87913344,POINT (175138.454 175085.460)
5,333784188_r_3,142621115,171304.537473,174685.576090,142621115,POINT (171304.537 174685.576)
6,3390195_0,1472593966,172670.028512,173730.286042,1472593966,POINT (172670.029 173730.286)
7,893781380_r_0-48247088_0,8306948508,173761.324240,173707.856064,8306948508,POINT (173761.324 173707.856)


### depots proximity

Calculate the mean distance between this unit (centroid) to each depots

In [127]:
''' Calculate the distance between the spatial unit centroid and each depot using gpd distance
    , and return the matrix '''
def cal_distance_matrix(unit_gdf: gpd.GeoDataFrame, depot_gdf: gpd.GeoDataFrame):
    distance_matrix = np.zeros((len(unit_gdf), len(depot_gdf)))
    for i, unit_centroid in enumerate(unit_gdf['geometry'].centroid):
        for j, depot in enumerate(depot_gdf['geometry']):
            distance_matrix[i, j] = unit_centroid.distance(depot)
    
    distance_matrix = pd.DataFrame(distance_matrix, index=unit_gdf['h3_index'], columns=depot_gdf['node_id'])
    
    distance_matrix['mean_distance'] = distance_matrix.mean(axis=1) / 1000  # Convert to km
    distance_matrix['median_distance'] = distance_matrix.median(axis=1) / 1000  # Convert to km
    distance_matrix = distance_matrix.reset_index()
    return distance_matrix

In [118]:
def agg_depots_proximity(spatial_unit_gdf: gpd.GeoDataFrame, depots_gdf: gpd.GeoDataFrame):
    new_gdf = spatial_unit_gdf.copy(deep=True)
    distance_matrix = cal_distance_matrix(new_gdf, depots_gdf)
    new_gdf = pd.merge(new_gdf, distance_matrix[['h3_index', 'mean_distance', 'median_distance']], on='h3_index')
    return new_gdf

In [181]:
for scenario, stats_gdf in scenario_stats_gdf.items():
    scenario_stats_gdf[scenario] = agg_depots_proximity(stats_gdf, depots_gdf)
    

### Depots number

In [136]:
def count_depots_in_unit(spatial_unit_gdf: gpd.GeoDataFrame, depots_gdf: gpd.GeoDataFrame):
    new_gdf = spatial_unit_gdf.copy(deep=True)
    sjoin_depots = gpd.sjoin(new_gdf, depots_gdf, how='left', predicate='contains')
    depot_stats = sjoin_depots.groupby('h3_index').agg({'node_id': 'count'}).reset_index().rename(columns={'node_id': 'depot_count'})
    # Merge the statistics with the spatial unit GeoDataFrame
    new_gdf = pd.merge(new_gdf, depot_stats, on='h3_index', how='left')
    return new_gdf

In [182]:
for scenario, stats_gdf in scenario_stats_gdf.items():
    scenario_stats_gdf[scenario] = count_depots_in_unit(stats_gdf, depots_gdf)

# Agg centrality metrics

In [148]:
centrality_root = r'../../../../data/clean/freightEmissions/networkCentrality/'
centralities_list = ['degreeCentrality', 'closeness_centrality', 'edges_betweeness', 'eigenvector_centrality']

In [173]:
def read_centrality(centrality_root: str, centrality_types: list, scenario: str):
    agg_centralities = {'node-based': None, 'link-based': None}
    if scenario.lower() == "cb":
        folder_kw = 'bikeNetwork'
    elif scenario.lower() == "van":
        folder_kw = 'carNetwork'
    elif scenario.lower() == "basic":
        folder_kw = 'carNetwork'
    else:
        raise ValueError('Invalid scenario name')
    
    for centrality_type in centrality_types:
        centrality_df = pd.read_csv(centrality_root + folder_kw + '//' + centrality_type + '.csv.gz', index_col=0)
        if centrality_type == 'edges_betweeness':
            agg_centralities['link-based'] = centrality_df
        else:
            if agg_centralities['node-based'] is None:
                agg_centralities['node-based'] = centrality_df
            else:
                agg_centralities['node-based'] = pd.merge(agg_centralities['node-based'], centrality_df, on='orig_id', how='left')
    agg_centralities['node-based'] = agg_centralities['node-based'].drop(columns=['node_x', 'node_y', 'node_id'])
    return agg_centralities
    

In [186]:
def agg_centralities(agg_centrality_dict: dict, 
                     nodes_gdf: gpd.GeoDataFrame,
                     links_gdf: gpd.GeoDataFrame,
                     scenario_stat_gdf: gpd.GeoDataFrame,
                     ):
    new_gdf = scenario_stat_gdf.copy(deep=True)

    nodes_with_centrality = nodes_gdf.merge(agg_centrality_dict['node-based'], left_on='node_id', right_on='orig_id', how='left')
    nodes_with_centrality = nodes_with_centrality.dropna()

    node_sjoin_stat = gpd.sjoin(new_gdf, nodes_with_centrality, how='left', predicate='contains')
    node_centrality_stat = node_sjoin_stat.groupby('h3_index')[['degree_centrality', 'closeness', 'eigenvector']].mean().reset_index()

    links_with_centrality = links_gdf.merge(agg_centrality_dict['link-based'], left_on='link_id', right_on='orig_id', how='left')
    links_with_centrality = links_with_centrality.dropna()

    link_sjoin_stat = gpd.sjoin(new_gdf, links_with_centrality, how='left', predicate='intersects')
    link_centrality_stat = link_sjoin_stat.groupby('h3_index')[['betweeness']].mean().reset_index()

    ''' merge stats with the scenario stat gdf '''
    new_gdf = pd.merge(new_gdf, node_centrality_stat, on='h3_index', how='left')
    new_gdf = pd.merge(new_gdf, link_centrality_stat, on='h3_index', how='left')

    return new_gdf

In [188]:
for scenario, stats_gdf in scenario_stats_gdf.items():
    centrality_dict = read_centrality(centrality_root, centralities_list, scenario)
    if scenario.lower() == 'cb':
        nodes_gdf = bike_nodes_gdf
        links_gdf = bike_network_gdf
    else:
        nodes_gdf = car_nodes_gdf
        links_gdf = car_network_gdf

    scenario_stats_gdf[scenario] = agg_centralities(centrality_dict, nodes_gdf, links_gdf, stats_gdf)

## Scale the centrality values

In [206]:
for scenario, stats_gdf in scenario_stats_gdf.items():
    stats_gdf['degree_centrality'] = stats_gdf['degree_centrality'] * 1e5
    stats_gdf['closeness'] = stats_gdf['closeness'] * 1e5
    stats_gdf['eigenvector'] = (stats_gdf['eigenvector'] - stats_gdf['eigenvector'].min()) / (stats_gdf['eigenvector'].max() - stats_gdf['eigenvector'].min())
    stats_gdf['betweeness'] = stats_gdf['betweeness'] * 1e5

# Output the stats_gdf

In [210]:
for scenario, stats_gdf in scenario_stats_gdf.items():
    stats_gdf.to_file(r'../../../../data/clean/freightEmissions/shp/stats/' + scenario + '_stats_h3Res' + str(h3_res) +'.geojson', driver='GeoJSON')

# Test